# Conversión entre Hexdigest y Plaintext
Como vamos a aplicar las operaciones con operaciones XOR entre bits, necesitamos codificar nuestro texto claro a un array de bytes.

In [ ]:
import binascii
password = b"apples are red"
print(password)

b'apples are red'


In [ ]:
passhex =  binascii.hexlify(password)

In [ ]:
print(passhex)

b'6170706c65732061726520726564'


In [ ]:
print(len(passhex))

28


Habrás notad oque el primer dìgito de la variable passhex es igual a 61; si nos fijamos en la tabla de caracteres ASCII. Para demostrar cómo convertir de caracter a hexadecimal

In [ ]:
print(ord('a'))

97


In [ ]:
print(hex(97))

TypeError: ignored

In [ ]:
print(format(ord('a'), 'x'))

61


La función unhexlify hace lo contrario y convierte un hexadecimal a un array de bits

In [ ]:
unpasshex = binascii.unhexlify(passhex)
unpasshex

b'apples are red'

Python permite que uses un caracter de escapa para convertir de hexadecimal a texto plano

In [ ]:
hx = '\x61'
hx

'a'

El proceso anterior tambien se puede hacer utilizando solamente las funciones nativas de Python

In [ ]:
password = "apples are red"
pswd_bits = password.encode()
print(pswd_bits)
print(list(pswd_bits))
pswd_bits_hex = pswd_bits.hex()
print(pswd_bits_hex)

b'apples are red'
[97, 112, 112, 108, 101, 115, 32, 97, 114, 101, 32, 114, 101, 100]
6170706c65732061726520726564


Y el proceso contrario

In [ ]:
pswd_bits = bytes.fromhex(pswd_bits_hex)
print(pswd_bits)
password = pswd_bits.decode()
print(password)

b'apples are red'
apples are red


# Implementando un cifrador de flujo
Vamos a implementar una solución que combinaa un bloque de un solo usao (OTP) con un generador seudoaleatorio cripográficamente seguro (CSPRNG). Y aplicarlo mediante una operación XOR para las operaciones de encriptación y desencriptación.
* Necesitamos intercambiar la clave entre el emisor y receptor como en todos los esquemas simétricos.
* La clave se puede usar sólo una vez.
* la longitud de la clave es igual a la longitud del mensaje.
Por el momento vamos a utilizar un generador sencillo para comprender el mecanismo y luego. 
Lo utilizaremos para encriptar caracteres ASCII, por lo que tenemos que pensar en bytes aleatorios en lugar de bits aleatorios.
La funcion crand() genera números de 31 bits, pero podemos 

In [ ]:
def crand(seed):
    r=[]
    r.append(seed)
    for i in range(30):
        r.append((16807*r[-1]) % 2147483647)
        if r[-1] < 0:
            r[-1] += 2147483647
    for i in range(31, 34):
        r.append(r[len(r)-31])
    for i in range(34, 344):
        r.append((r[len(r)-31] + r[len(r)-3]) % 2**32)
    while True:
        next = r[len(r)-31]+r[len(r)-3] % 2**32
        r.append(next)
        yield (next >> 1 if next < 2**32 else (next % 2**32) >> 1)

mygen = crand(2018)
firstfour = [next(mygen) for i in range(4)]
print(firstfour)

[1471611625, 1204518815, 463882823, 963005816]


In [ ]:
rands = [next(mygen) for i in range(4)]
plaintext = b"Hello world!"

hexplain = binascii.hexlify(plaintext)
hexkey = "".join(map(lambda x: format(x, 'x')[-6:], rands))

cipher_as_int = int(hexplain, 16) ^ int(hexkey, 16)
cipher_as_hex = format(cipher_as_int, 'x')

cipher_as_hex

'a6f2eb3c80552a4fb0c9773c'

In [ ]:
cipher_as_int = int(cipher_as_hex, 16)
cipher_as_int

plain_text_int = cipher_as_int ^ int(hexkey, 16)
plain_text_hex = format(plain_text_int, 'x')
plain_text = binascii.unhexlify(plain_text_hex)

plain_text

b'Hello world!'

En resumen se está haciendo una operación a nivel de bits, mas o menos así

In [ ]:
mensaje = 12345364567586754756745675475685467
print("Mensaje  :", format(mensaje, 'b'))
s = 254754786786786786786786786785564
print("Sec.seudo:",format(s, 'b'))
C = mensaje ^ s
print("Cifrado  :",format(C,'b'))
M = C^s
print("Descifrado  :",format(M,'b'))